In [25]:
import nltk
import pandas as pd
#from sklearn.linear_model import LogisticRegression

In [26]:
# Reading in the train data into a dataframe
kaggle_movie_reviews_df_original = pd.read_csv('C:/Users/daedr/OneDrive/Documents/Masters Courses/IT/IST664/FinalProject/FinalProjectData/FinalProjectData/kagglemoviereviews/corpus/train.tsv', sep='\t')


In [24]:
# defining a method to get tokenize a sentence.
def get_tokens(str):
    tokens = nltk.word_tokenize(str) 
    tokens = [w.lower( ) for w in tokens]
    return tokens

In [22]:
def vectorize_feature_documents(feature):
    vectorized_feature_document = []
    for document in feature:
        vectorized_feature_document.append(document[0])
    return vectorized_feature_document

In [23]:
def vectorize_feature_rating(feature):
    vectorized_feature_rating = []
    for rating in feature:
        vectorized_feature_rating.append(rating[1])
    return vectorized_feature_rating

In [275]:
def get_key_dictionary(dictionary, key_dictionary):
    for key in dictionary:
        if key not in key_dictionary.keys():
            key_dictionary[key] = ''
    return key_dictionary

In [276]:
def get_all_keys_in_feature(feature):
    key_dictionary = {}
    for dictionary in feature:
        get_key_dictionary(dictionary, key_dictionary)
    return key_dictionary

In [277]:
def get_default_vectorization_list(all_existing_keys):
    i = 0
    default_vectorized_list = []
    while i < len(all_existing_keys):
        default_vectorized_list.append(0)
        i += 1
    return default_vectorized_list

In [301]:
# defining method to get a number of the most common used words
def most_common_used_words(method_phrases, num_want):
    kaggler_movie_reviews_tokens = []
    i = 0
    while i < len(method_phrases):
        kaggler_movie_reviews_tokens.extend(get_tokens(method_phrases[i]))
        i = i + 1
    distributed_words = nltk.FreqDist(kaggler_movie_reviews_tokens)
    most_used_words = distributed_words.most_common(num_want)
    return most_used_words

In [74]:
## BAG OF WORDS WITH LEMMATIZATION

In [284]:
# converting a list of string back to a single string 
def rejoin_string(list_strings):
    return " ".join(list_strings)

In [285]:
def convert_list_of_list_to_list_of_string(list_of_list):
    new_list = []
    i = 0
    while i < len(list_of_list):
        new_list.append(rejoin_string(list_of_list[i]))
        i = i + 1
    return new_list

In [286]:
# Preprocessing/filtering of the data Lemmatization
from nltk.corpus import wordnet
phrases = kaggle_movie_reviews_df_original["Phrase"].to_list()
lemmatizer = nltk.WordNetLemmatizer()
i = 0
phrases_lemmatized = []
while i < len(phrases):
    str = get_tokens(phrases[i])
    phrase_lemmatized = [ lemmatizer.lemmatize(word) for word in str ]
    phrases_lemmatized.append(phrase_lemmatized)
    i = i + 1

In [287]:
vocabulary = set()
bag_of_words = []

i = 0
while i < len(phrases_lemmatized):
    counts = {}
    vocabulary.update(phrases_lemmatized[i])
    for token in phrases_lemmatized[i]:
        if token in counts:
            counts[token] += 1
        else:
            counts[token] = 1
    bag_of_words.append(counts)
    i += 1



In [288]:
sentiment = kaggle_movie_reviews_df_original["Sentiment"].to_list()

In [289]:
feature_set = []
i = 0
while i < len(bag_of_words):
    feature_set.append( (bag_of_words[i], sentiment[i]) )
    i += 1

In [81]:
# split data into a training set and a test set, using a 90%/10% split
size = int(len(feature_set) * 0.1)
train_set, test_set = feature_set[size:], feature_set[:size]

In [82]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [83]:
print(nltk.classify.accuracy(classifier, test_set))

0.3892733564013841


In [10]:
## cross-validation ##
# this function takes the number of folds, the feature sets
# it iterates over the folds, using different sections for training and testing in turn
# it prints the accuracy for each fold and the average accuracy at the end
def cross_validation_accuracy(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    print('Each fold size:', subset_size)
    accuracy_list = []
    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i + 1) * subset_size):]
        # train using train_this_round
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        # evaluate against teset_this_round save accuracy
        accuracy_this_round = nltk.classify.accuracy(classifier, test_this_round)
        print(i, accuracy_this_round)
        accuracy_list.append(accuracy_this_round)
    # find mean accuracy over all rounds
    print('mean accuracy', sum(accuracy_list) / num_folds)

In [85]:
# perform the cross-validation on the featuresets with word features and generate accuracy
num_folds = 10
cross_validation_accuracy(num_folds, feature_set)

Each fold size: 15606
0 0.3892733564013841
1 0.3602460592079969
2 0.3571062411892862
3 0.342240164039472
4 0.33871587850826607
5 0.33384595668332695
6 0.330642060745867
7 0.32320902217095987
8 0.3307061386646162
9 0.3229527104959631
mean accuracy 0.34289375881071393


In [86]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [11]:
# Fucntion to compute precision, recall and F1 for each label
# and for any number of labels
# Input: list of gold labels, list of predicted labels (in same order)
# Output: prints precision, recall and F1 for each label
def eval_measures(gold, predicted):
    # get a list of labels
    labels = set(gold)
    labels = [elem for elem in labels]
    # these lists have values for each label
    recall_list = []
    precision_list = []
    F1_list = []
    for lab in labels:
        # for each label, compare gold and predicted lists and compute values
        TP = FP = FN = TN = 0
        for i, val in enumerate(gold):
            if val == lab and predicted[i] == lab: TP += 1
            if val == lab and predicted[i] != lab: FN += 1
            if val != lab and predicted[i] == lab: FP += 1
            if val != lab and predicted[i] != lab: TN += 1
        # use these to compute recall, precision, F1
        if TP != 0 or FP != 0:
            recall = TP / (TP + FP)
        else:
            recall = 0
        precision = TP / (TP + FN)
        recall_list.append(recall)
        precision_list.append(precision)
        if recall != 0 or precision != 0:
            F1_list.append( 2 * (recall * precision) / (recall + precision))
        else:
            F1_list.append(0)
    # the evaluation measures in a table with one row per label
    print('\tPrecision\tRecall\t\tF1')
    # print measures for each label
    for i, lab in enumerate(labels):
        print(lab, '\t', "{:10.3f}".format(precision_list[i]), \
              "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))

In [88]:
# call the function with our data
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.687      0.134      0.224
1 	      0.339      0.274      0.303
2 	      0.363      0.824      0.504
3 	      0.397      0.333      0.362
4 	      0.592      0.221      0.322


In [ ]:
# SciKit BAG OF WORDS 

In [ ]:
# Base functions for BOW SciKit

In [306]:
def convert_list_to_dictionary(key_list):
    key_dictionary = {}
    for item in key_list:
        key_dictionary[item] = ''
    return key_dictionary

In [321]:
def convert_dictionary_to_scikit_usable_list_uneven_lists_BOW(dictionary, all_existing_keys):
    usable_list = get_default_vectorization_list(all_existing_keys)
    for key in dictionary:
        if key in all_existing_keys:
            usable_list[list(all_existing_keys.keys()).index(key)] = dictionary[key]
    return usable_list

In [322]:
def convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_BOW(list_dictionary):
    usable = []
    keys_for_bag_of_words = most_common_used_words(phrases, 1000)
    keys_for_bag_of_words = convert_list_to_dictionary(keys_for_bag_of_words)
    for dictionary in list_dictionary:
        usable.append(convert_dictionary_to_scikit_usable_list_uneven_lists_BOW(dictionary, keys_for_bag_of_words))
    return usable  

In [323]:
# Regression

In [324]:
from sklearn.linear_model import LogisticRegression

In [325]:
vectorized_documents = vectorize_feature_documents(feature_set)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_BOW(vectorized_documents)

In [326]:
vectorized_rating = vectorize_feature_rating(feature_set)

In [327]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [328]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [329]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [330]:
clf.fit(train_predictors, train_rating)

LogisticRegression(multi_class='ovr')

In [331]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5577342047930284


In [299]:
# Neural Network

In [332]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [333]:
from sklearn.metrics import accuracy_score

In [334]:
accuracy_score(test_rating, y_pred)

0.5577342047930284

In [335]:
# Dependency Tree

In [336]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5577342047930284


In [19]:
## BAG OF WORDS NO FILTERING

In [352]:
vocabulary = set()
bag_of_words = []

i = 0
while i < len(phrases):
    new_phrases = get_tokens(phrases[i])
    counts = {}
    vocabulary.update(new_phrases)
    for token in new_phrases:
        if token in counts:
            counts[token] += 1
        else:
            counts[token] = 1
    bag_of_words.append(counts)
    i += 1



In [353]:
feature_set = []
i = 0
while i < len(bag_of_words):
    feature_set.append( (bag_of_words[i], sentiment[i]) )
    i += 1

In [23]:
# split data into a training set and a test set, using a 90%/10% split
size = int(len(feature_set) * 0.1)
train_set, test_set = feature_set[size:], feature_set[:size]

In [24]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [25]:
print(nltk.classify.accuracy(classifier, test_set))

0.38767140843265413


In [26]:
num_folds = 10
cross_validation_accuracy(num_folds, feature_set)

Each fold size: 15606
0 0.38767140843265413
1 0.357362552864283
2 0.356080994489299
3 0.3410867615019864
4 0.34371395617070355
5 0.3311546840958606
6 0.3304498269896194
7 0.3246828143021915
8 0.33211585287709855
9 0.32436242470844545
mean accuracy 0.34286812764321417


In [27]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [28]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.685      0.134      0.224
1 	      0.337      0.270      0.300
2 	      0.361      0.822      0.502
3 	      0.396      0.331      0.360
4 	      0.592      0.222      0.323


In [ ]:
# Bag of Word no Filtering SciKit

In [ ]:
# Reggression

In [354]:
from sklearn.linear_model import LogisticRegression

In [355]:
vectorized_documents = vectorize_feature_documents(feature_set)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_BOW(vectorized_documents)

In [356]:
vectorized_rating = vectorize_feature_rating(feature_set)

In [357]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [358]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [359]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [360]:
clf.fit(train_predictors, train_rating)

LogisticRegression(multi_class='ovr')

In [361]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5577342047930284


In [ ]:
# Neural Network

In [347]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [348]:
from sklearn.metrics import accuracy_score

In [349]:
accuracy_score(test_rating, y_pred)

0.5577342047930284

In [350]:
# Dependency Tree

In [351]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5577342047930284


In [29]:
# SUBJECTIVITY METHODS

In [12]:
# this function returns a dictionary where you can look up words and get back 
#     the four items of subjectivity information described above
def readSubjectivity(path):
    flexicon = open(path, 'r')
    # initialize an empty dictionary
    sldict = { }
    for line in flexicon:
        fields = line.split()   # default is to split on whitespace
        # split each field on the '=' and keep the second part as the value
        strength = fields[0].split("=")[1]
        word = fields[2].split("=")[1]
        posTag = fields[3].split("=")[1]
        stemmed = fields[4].split("=")[1]
        polarity = fields[5].split("=")[1]
        if (stemmed == 'y'):
            isStemmed = True
        else:
            isStemmed = False
        # put a dictionary entry with the word as the keyword
        #     and a list of the other values
        sldict[word] = [strength, posTag, isStemmed, polarity]
    return sldict

In [13]:
SLpath = "C:/Users/daedr/OneDrive/Documents/Masters Courses/IT/IST664/WK8/subjclueslen1-hltemnlp05.tff/subjclueslen1-HLTEMNLP05.tff"

In [14]:
# import the Subjectivity program as a module to use the function
#import Subjectivity
SL = readSubjectivity(SLpath)

In [15]:
# define features that include word counts of subjectivity words
# negative feature will have number of weakly negative words +
# positive feature has similar definition
# not counting neutral words
def SL_features(document, word_features, SL):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    # count variable for the 4 classes of subjectivity
    weakPos = 0
    strongPos = 0
    weakNeg = 0
    strongNeg = 0
    for word in document_words:
        if word is SL:
            strength, posTag, isStemmed, polarity = SL[word]
            if strength == 'waksubj' and polarity == 'positive':
                weakPos += 1
            if strength == 'strongsubj' and polarity == 'positive':
                strongPos += 1
            if strength == 'weaksubj' and polarity == 'negative':
                weakNeg += 1
            if strength == 'strongsubj' and polarity == 'negative':
                strongNeg += 1
            features['positivecount'] = weakPos + (2 * strongPos)
            features['negativecount'] = weakNeg + (2 * strongNeg)
    return features

In [16]:
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    return features

In [18]:
def get_only_words_from_tuple(tuple_list):
    i = 0
    words_only_list = []
    while i < len(tuple_list):
        words_only_list.append(tuple_list[i][0])
        i = i + 1
    return words_only_list

In [37]:
# SUBJECTIVITY ANALYSIS LEMMATIZATION

In [30]:
lemmatization_pure_string = convert_list_of_list_to_list_of_string(phrases_lemmatized)
most_used_words_tuple = most_common_used_words(lemmatization_pure_string, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(phrases_lemmatized):
    document.append( (phrases_lemmatized[i], sentiment[i]) )
    i += 1

In [31]:
SL_featuresets = [(SL_features(d, most_used_words_list, SL), c) for (d, c) in document]

In [ ]:
#PRINTING FOR TESTING PURPOSES CAN DELETE AFTERWORDS

In [45]:
size = int(len(SL_featuresets) * 0.1)
train_set, test_set = SL_featuresets[size:], SL_featuresets[:size]

In [46]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5742022299115724


In [47]:
# I'm unable to do this with the full set. It causes the machine to crash so used 10,000 elements of the full set
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.5636881452474191
1 0.5238875044499822
2 0.5250978996084016
3 0.5317906728373086
4 0.5229619081523674
5 0.51812032751869
6 0.5139907440370238
7 0.5173371306514774
8 0.5139195443218227
9 0.5060875756496974
mean accuracy 0.5236881452474191


In [154]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [153]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.687      0.134      0.224
1 	      0.339      0.274      0.303
2 	      0.363      0.824      0.504
3 	      0.397      0.333      0.362
4 	      0.592      0.221      0.322


In [ ]:
# SciKit Learning Subjectivity Analysis Feature Base Functions

In [19]:
def convert_dictionary_to_scikit_usable_list(dictionary):
    usable_list = []
    for key in dictionary:
        if dictionary[key] == True:
            usable_list.append(1)
        else:
            usable_list.append(0)
    return usable_list

In [20]:
def convert_list_of_dictionary_to_scikit_usable_list_Subjectivity(list_dictionary):
    usable_list = []
    for dictionary in list_dictionary:
        usable_list.append(convert_dictionary_to_scikit_usable_list(dictionary))
    return usable_list
                        

In [ ]:
# SciKit learning Subjectivity Analysis feature Lemmatization

In [ ]:
# Regression

In [28]:
from sklearn.linear_model import LogisticRegression

In [32]:
vectorized_documents = vectorize_feature_documents(SL_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Subjectivity(vectorized_documents)

In [27]:
vectorized_rating = vectorize_feature_rating(SL_featuresets)

In [28]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [29]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [32]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [33]:
clf.fit(train_predictors, train_rating)

C:\Users\daedr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

In [34]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.593041138023837


In [ ]:
# Neaural Network

In [38]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(test_rating, y_pred)

0.5874022811739075

In [41]:
# Decision Tree

In [37]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5388952966807639


In [ ]:
# SUBJECTIVITY ANALYSIS ON ORIGINAL DATA

In [36]:
most_used_words_tuple = most_common_used_words(phrases, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(phrases):
    document.append( (phrases[i], sentiment[i]) )
    i += 1

In [37]:
SL_featuresets = [(SL_features(d, most_used_words_list, SL), c) for (d, c) in document]

In [35]:
# split data into a training set and a test set, Didn't use full size. It was too large and caused my machine to crash. Was not possible on my laptop.
size = int(len(SL_featuresets) * 0.1)
train_set, test_set = SL_featuresets[size:], SL_featuresets[:size]

In [170]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5587594514930155


In [171]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.5471698113207547
1 0.5072979708081168
2 0.5056603773584906
3 0.5141331434674261
4 0.49911000355998575
5 0.501317194731221
6 0.4993236027055892
7 0.49676041295834816
8 0.49241723033107865
9 0.4937700249199003
mean accuracy 0.5056959772160912


In [172]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [173]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.028      0.340      0.052
1 	      0.082      0.281      0.127
2 	      0.939      0.601      0.733
3 	      0.108      0.266      0.153
4 	      0.000      0.000      0.000


In [ ]:
# SciKit learning Subjectivity Analysis feature Original Data

In [ ]:
# Regression

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
vectorized_documents = vectorize_feature_documents(SL_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Subjectivity(vectorized_documents)

In [40]:
vectorized_rating = vectorize_feature_rating(SL_featuresets)

In [41]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [42]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [43]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [44]:
clf.fit(train_predictors, train_rating)

LogisticRegression(multi_class='ovr')

In [45]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5601050877867487


In [ ]:
# Neural Network

In [46]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
accuracy_score(test_rating, y_pred)

0.5601050877867487

In [ ]:
# Decision Tree

In [49]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5595924644367551


In [ ]:
# NEGATION FEATURES GENERAL FUNCTIONS

In [50]:
# this list of negation words includes some "approximate negators" like hardly and rarely
negationwords = [ 'no', 'not', 'never', 'none', 'nowhere', 'nothing', 'noone', 'rather', 'hardly', 'scarcely', 'rarely', 'seldom', 'neither', 'nor']

In [51]:
def NOT_features(document, word_features, negationwords):
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = False
        features['V_NOT{}'.format(word)] = False
    # go through document words in order
    for i in range(0, len(document)):
        word = document[i]
        if ((i + 1) < len(document)) and ((word in negationwords) or (word.endswith("n't"))):
            i += 1
            features['V_NOT{}'.format(document[i])] = (document[i] in word_features)
        else:
            features['V_{}'.format(word)] = (word in word_features)
    return features

In [11]:
# NEGATION WORDS WITH LEMMATIZATION

In [362]:
lemmatization_pure_string = convert_list_of_list_to_list_of_string(phrases_lemmatized)
most_used_words_tuple = most_common_used_words(lemmatization_pure_string, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

In [363]:
document = []
i = 0
while i < len(lemmatization_pure_string):
    document.append( (lemmatization_pure_string[i], sentiment[i]) )
    i += 1

In [364]:
# define the feature sets
NOT_featuresets = [(NOT_features(d, most_used_words_list, negationwords), c) for (d, c) in document]

In [180]:
size = int(len(NOT_featuresets) * 0.1)
train_set, test_set = NOT_featuresets[size:], NOT_featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.36742278610790724

In [181]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.33300106799572804
1 0.31391954432182273
2 0.2943396226415094
3 0.28892844428622283
4 0.2798860804556782
5 0.27739409042363833
6 0.2633677465290139
7 0.257315770736917
8 0.27020291918832323
9 0.2512637949448202
mean accuracy 0.28296190815236744


In [182]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [183]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.401      0.092      0.149
1 	      0.145      0.207      0.170
2 	      0.467      0.752      0.576
3 	      0.286      0.218      0.248
4 	      0.263      0.106      0.151


In [ ]:
# SciKit Vectorization for Negation

In [273]:
def convert_dictionary_to_scikit_usable_list_uneven_lists(dictionary, all_existing_keys):
    usable_list = get_default_vectorization_list(all_existing_keys)
    for key in dictionary:
        if dictionary[key] == True:
            usable_list[list(all_keys.keys()).index(key)] = 1
    return usable_list

In [274]:
def convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists(list_dictionary):
    usable_list = []
    all_existing_keys = get_all_keys_in_feature(list_dictionary)
    for dictionary in list_dictionary:
        usable_list.append(convert_dictionary_to_scikit_usable_list_uneven_lists(dictionary, all_existing_keys))
    return usable_list                    

In [ ]:
# SciKit NEGATION Lemmatization

In [ ]:
#Regression

In [142]:
from sklearn.linear_model import LogisticRegression

In [143]:
vectorized_documents = vectorize_feature_documents(NOT_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists(vectorized_documents)

In [148]:
vectorized_rating = vectorize_feature_rating(NOT_featuresets)

In [149]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [150]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [151]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [152]:
clf.fit(train_predictors, train_rating)

LogisticRegression(multi_class='ovr')

In [153]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5606177111367423


In [ ]:
# Neural Network

In [154]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [155]:
from sklearn.metrics import accuracy_score

In [156]:
accuracy_score(test_rating, y_pred)

0.5612584903242342

In [157]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5602332436242471


In [ ]:
# NEGATION WORDS WITH WITHOUT FILTERING

In [158]:
most_used_words_tuple = most_common_used_words(phrases, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

In [159]:
document = []
i = 0
while i < len(phrases):
    document.append( (phrases[i], sentiment[i]) )
    i += 1

In [160]:
# define the feature sets
NOT_featuresets = [(NOT_features(d, most_used_words_list, negationwords), c) for (d, c) in document]

In [187]:
size = int(len(NOT_featuresets) * 0.1)
train_set, test_set = NOT_featuresets[size:], NOT_featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.3362809175957965

In [188]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.2965468138127447
1 0.2830188679245283
2 0.26407974368102527
3 0.25546457814168744
4 0.25026699893200427
5 0.24514061943752224
6 0.23353506585973657
7 0.22954788180847277
8 0.2387326450694197
9 0.22819508721965112
mean accuracy 0.25245283018867926


In [189]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [190]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.450      0.092      0.153
1 	      0.161      0.198      0.177
2 	      0.401      0.766      0.527
3 	      0.290      0.218      0.249
4 	      0.275      0.100      0.146


In [ ]:
# SciKit for NEGATION WORDS WITH WITHOUT FILTERING

In [161]:
from sklearn.linear_model import LogisticRegression

In [162]:
vectorized_documents = vectorize_feature_documents(NOT_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists(vectorized_documents)

In [163]:
vectorized_rating = vectorize_feature_rating(NOT_featuresets)

In [164]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [165]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [166]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [167]:
clf.fit(train_predictors, train_rating)

LogisticRegression(multi_class='ovr')

In [168]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5601050877867487


In [ ]:
# Neural Network

In [169]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [170]:
from sklearn.metrics import accuracy_score

In [171]:
accuracy_score(test_rating, y_pred)

0.5601050877867487

In [172]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5595924644367551


In [ ]:
# LIWC Sentiment analysis 

In [ ]:
### AS A NOTE. LIWC IS REQUIRED TO BE INSTALLED BEFORE EXECUTING THE BELOW CODE. 

In [173]:
import re

In [174]:
import liwc
from collections import Counter

In [175]:
parse, category_names = liwc.load_token_parser('C:/Users/daedr/OneDrive/Documents/Masters Courses/IT/IST664/FinalProject/LIWC2007_English100131.dic')

In [176]:
def liwc_features(parse, category_names, phrases_tokens_list):
    liwc_sentiment_analysis_complete = []
    i = 0
    for phrase in phrases_tokens_list:
        liwc_sentiment_analysis_phrase = Counter(category for token in phrase for category in parse(token.lower()))
        liwc_sentiment_analysis_complete.append((dict(liwc_sentiment_analysis_phrase), sentiment[i]))
        i += 1
    return liwc_sentiment_analysis_complete

In [177]:
# LIWC Sentiment Analysis Lemmatization

In [178]:
liwc_feature = liwc_features(parse, category_names, phrases_lemmatized)

In [27]:
size = int(len(liwc_feature) * 0.1)
train_set, test_set = liwc_feature[size:], liwc_feature[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.40516468025118546

In [31]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.380064079743681
1 0.36333214667141334
2 0.3572089711641153
3 0.34524741901032396
4 0.3574937700249199
5 0.34937700249199005
6 0.3292986828052688
7 0.3263794944820221
8 0.32032751868992526
9 0.31363474546101816
mean accuracy 0.3442363830544678


In [32]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [33]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.583      0.091      0.157
1 	      0.209      0.240      0.223
2 	      0.498      0.755      0.600
3 	      0.315      0.315      0.315
4 	      0.217      0.177      0.195


In [ ]:
# LIWC Sentiment Analysis Base Functions

In [180]:
print(category_names)

['funct', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 'ipron', 'article', 'verb', 'auxverb', 'past', 'present', 'future', 'adverb', 'preps', 'conj', 'negate', 'quant', 'number', 'swear', 'social', 'family', 'friend', 'humans', 'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'cogmech', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'inhib', 'incl', 'excl', 'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', 'relativ', 'motion', 'space', 'time', 'work', 'achieve', 'leisure', 'home', 'money', 'relig', 'death', 'assent', 'nonfl', 'filler']


In [241]:
def convert_dictionary_to_scikit_usable_list_uneven_lists_LIWC(dictionary, all_existing_keys):
    usable_list = get_default_vectorization_list(all_existing_keys)
    for key in dictionary:
        usable_list[list(all_existing_keys.keys()).index(key)] = dictionary[key]
    return usable_list

In [242]:
def convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_LIWC(list_dictionary):
    usable = []
    all_existing_keys = get_all_keys_in_feature(list_dictionary)
    for dictionary in list_dictionary:
        usable.append(convert_dictionary_to_scikit_usable_list_uneven_lists_LIWC(dictionary, all_existing_keys))
    return usable  

In [ ]:
# LIWC Sentiment Analysis Lemmatization SciKit 

In [244]:
from sklearn.linear_model import LogisticRegression

In [245]:
vectorized_documents = vectorize_feature_documents(liwc_feature)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_LIWC(vectorized_documents)

In [246]:
vectorized_rating = vectorize_feature_rating(NOT_featuresets)

In [247]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [248]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [249]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [250]:
clf.fit(train_predictors, train_rating)

C:\Users\daedr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

In [251]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5742663078303216


In [ ]:
# Neural Network

In [252]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [253]:
from sklearn.metrics import accuracy_score

In [254]:
accuracy_score(test_rating, y_pred)

0.5929129821863386

In [ ]:
# Decision Tree

In [255]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5401127771369986


In [ ]:
# LIWC Sentiment Analysis Original data set

In [256]:
tokenized_phrases = []
for phrase in phrases:
    tokenized_phrases.append(get_tokens(phrase))

In [257]:
liwc_feature = liwc_features(parse, category_names, tokenized_phrases)

In [40]:
size = int(len(liwc_feature) * 0.1)
train_set, test_set = liwc_feature[size:], liwc_feature[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.4097782904011278

In [41]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.3839088643645425
1 0.3656105375578498
2 0.3589177643289427
3 0.3505873976504094
4 0.3607689569241723
5 0.35172659309362764
6 0.3289426842292631
7 0.33072267710929154
8 0.3187611249555002
9 0.31434674261302953
mean accuracy 0.34642933428266287


In [42]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

In [43]:
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.594      0.092      0.159
1 	      0.207      0.247      0.225
2 	      0.508      0.751      0.606
3 	      0.309      0.318      0.313
4 	      0.226      0.185      0.204


In [ ]:
# LIWC Sentiment Analysis Lemmatization SciKit 

In [258]:
from sklearn.linear_model import LogisticRegression

In [278]:
vectorized_documents = vectorize_feature_documents(liwc_feature)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_LIWC(vectorized_documents)

In [279]:
vectorized_rating = vectorize_feature_rating(NOT_featuresets)

In [280]:
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]

In [281]:
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]

In [263]:
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')

In [264]:
clf.fit(train_predictors, train_rating)

C:\Users\daedr\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

In [265]:
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5756760220428041


In [ ]:
# Neural Network

In [266]:
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

In [267]:
from sklearn.metrics import accuracy_score

In [268]:
accuracy_score(test_rating, y_pred)

0.5894527745738818

In [269]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5347943098808151


In [ ]:
# SUBJECTIVITY and NEGATION Features combined Lemmatization

In [372]:
lemmatization_pure_string = convert_list_of_list_to_list_of_string(phrases_lemmatized)
most_used_words_tuple = most_common_used_words(lemmatization_pure_string, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(phrases_lemmatized):
    document.append( (phrases_lemmatized[i], sentiment[i]) )
    i += 1

SL_featuresets = [(SL_features(d, most_used_words_list, SL), c) for (d, c) in document]

In [373]:
lemmatization_pure_string = convert_list_of_list_to_list_of_string(phrases_lemmatized)
most_used_words_tuple = most_common_used_words(lemmatization_pure_string, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(lemmatization_pure_string):
    document.append( (lemmatization_pure_string[i], sentiment[i]) )
    i += 1

NOT_featuresets = [(NOT_features(d, most_used_words_list, negationwords), c) for (d, c) in document]

In [376]:
i = 0
while i < len(SL_featuresets):
    SL_featuresets[i][0].update(NOT_featuresets[i][0])
    i += 1

In [378]:
size = int(len(SL_featuresets) * 0.1)
train_set, test_set = SL_featuresets[size:], SL_featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.36742278610790724

In [379]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.33300106799572804
1 0.31391954432182273
2 0.2943396226415094
3 0.28892844428622283
4 0.2798860804556782
5 0.27739409042363833
6 0.2633677465290139
7 0.257315770736917
8 0.27020291918832323
9 0.2512637949448202
mean accuracy 0.28296190815236744


In [380]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.401      0.092      0.149
1 	      0.145      0.207      0.170
2 	      0.467      0.752      0.576
3 	      0.286      0.218      0.248
4 	      0.263      0.106      0.151


In [381]:
# Regression
from sklearn.linear_model import LogisticRegression

vectorized_documents = vectorize_feature_documents(SL_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_LIWC(vectorized_documents)
vectorized_rating = vectorize_feature_rating(SL_featuresets)
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')
clf.fit(train_predictors, train_rating)
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5606177111367423


In [382]:
# Neural Network
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

from sklearn.metrics import accuracy_score

accuracy_score(test_rating, y_pred)

0.5609381007304882

In [383]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5602332436242471


In [ ]:
# SUBJECTIVITY and NEGATION Features combined Original Dataset

In [384]:
most_used_words_tuple = most_common_used_words(phrases, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(phrases_lemmatized):
    document.append( (phrases_lemmatized[i], sentiment[i]) )
    i += 1

SL_featuresets = [(SL_features(d, most_used_words_list, SL), c) for (d, c) in document]

In [385]:
most_used_words_tuple = most_common_used_words(phrases, 500)
most_used_words_list = get_only_words_from_tuple(most_used_words_tuple)

document = []
i = 0
while i < len(lemmatization_pure_string):
    document.append( (lemmatization_pure_string[i], sentiment[i]) )
    i += 1

NOT_featuresets = [(NOT_features(d, most_used_words_list, negationwords), c) for (d, c) in document]

In [386]:
i = 0
while i < len(SL_featuresets):
    SL_featuresets[i][0].update(NOT_featuresets[i][0])
    i += 1

In [387]:
size = int(len(SL_featuresets) * 0.1)
train_set, test_set = SL_featuresets[size:], SL_featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.3402537485582468

In [388]:
num_folds = 10
cross_validation_accuracy(num_folds, train_set)

Each fold size: 14045
0 0.3013171947312211
1 0.2865076539693841
2 0.2672125311498754
3 0.25788536845852617
4 0.2533997864008544
5 0.24727661089355643
6 0.23645425418298327
7 0.2296190815236739
8 0.24279102883588466
9 0.22862228551085795
mean accuracy 0.2551085795656817


In [389]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    goldlist.append(label)
    predictedlist.append(classifier.classify(features))

eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
0 	      0.436      0.093      0.154
1 	      0.163      0.195      0.177
2 	      0.407      0.761      0.530
3 	      0.294      0.216      0.249
4 	      0.275      0.106      0.153


In [390]:
# Regression
from sklearn.linear_model import LogisticRegression

vectorized_documents = vectorize_feature_documents(SL_featuresets)
vectorized_documents = convert_list_of_dictionary_to_scikit_usable_list_Uneven_Lists_LIWC(vectorized_documents)
vectorized_rating = vectorize_feature_rating(SL_featuresets)
size = int(len(vectorized_documents) * 0.1)
train_predictors, test_predictors = vectorized_documents[size:], vectorized_documents[:size]
size = int(len(vectorized_rating) * 0.1)
train_rating, test_rating = vectorized_rating[size:], vectorized_rating[:size]
clf = LogisticRegression(C=1.0, solver='lbfgs', multi_class='ovr')
clf.fit(train_predictors, train_rating)
score = clf.score(test_predictors, test_rating)
print("Accuracy:", score)

Accuracy: 0.5602332436242471


In [391]:
# Neural Network
import sklearn.neural_network 

neural_network = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu')
neural_network.fit(train_predictors, train_rating) 
y_pred = neural_network.predict(test_predictors)

from sklearn.metrics import accuracy_score

accuracy_score(test_rating, y_pred)

0.5602973215429963

In [392]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_predictors, train_rating)
y_pred = clf.predict(test_predictors)
accuracy = accuracy_score(test_rating, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.5595283865180058
